# Project Journal

Name: Erin Widener

Research Question: In considering 3 different energy production sources (total fossil fuel production, nuclear electric power production, total renewable energy production) and 1 energy import variable (primary energy imports), can at least one of these sources of energy "inputs" accurately predict total primary energy consumption?  

Variables: Response = Total Primary Energy Consumption (Y)
           Predictor(s) = Total Fossil Fuel Production (B1, X1), Nuclear Electric Power Production (B2, X2), Total Renewable Energy Production (B3, X3), Primary Energy Imputs (B4, X4)

Hypothesis: Is there at least one predictor useful in predicting total primary energy consumption? 

Ho: B1 = B2 = B3 = B4 = 0 
Ha: At least one Bj ≠ 0 (j= 1-4) 

## Data Prep & EDA

### Data Cleaning Summary

**Summary of data cleaning process:**
1. Step 1: Set wd and install/load packages
2. Step 2: Import data
3. Step 3: Asses the structure of the data
4. Step 4: Build dataframe with relevant variables

**Issues Encountered and Resolutions:**
Some minor loading issues with some packages (ALSM) and extracting relevant variables from dataset renaming them to analysis friendly terms like "Fossil" or "Renewable"


In [ ]:
##Step 1 - set working directory and install/load packages (installation code deleted to show clearly the packeges loaded from library)
setwd("C:/Users/Hunte/OneDrive - purdue.edu/STAT 512/Final Project")
library(vroom)
library(tidyverse)
library(IRkernel)
library(ALSM)
library(car)
library(fmsb)
library(GGally)
library(lmridge)
library(lmtest)
library(MASS)
library(glmnet) 
library(boot) 
library(caret)

In [ ]:
##Step 2 - Import dataset
dataset <- read.csv("World Energy Overview.csv")
matrix.data <- as.matrix(dataset)

In [ ]:
##Step 3 - Asses the structure of the data
str(dataset)
str(matrix.data)

In [ ]:
#Step 4 Building dataframe with relevant variables
Y <- dataset$Total.Primary.Energy.Consumption 
X1 <- dataset$Total.Fossil.Fuels.Production
X2 <- dataset$Nuclear.Electric.Power.Production
X3 <- dataset$Total.Renewable.Energy.Production 
X4 <- dataset$Primary.Energy.Imports

predictor.response.data <- data.frame("Consumption"= Y, "Fossil" = X1, 
                                                 "Nuclear" = X2, 
                                                 "Renewable" = X3, 
                                                 "Imports" = X4)
head(predictor.response.data)

### Summary Statistics

In [ ]:
#summary statistics 
mean(Y)
sd(Y)
mean(X1)
sd(X1)
mean(X2)
sd(X2)
mean(X3)
sd(X3)
mean(X4)
sd(X4)

### Exploratory Data Analysis Findings

The main finding of this analysis was that the scale of each variable is quite different. This may make remedial measures such as log transformations have a risk of overfitting or showing as more powerful than they really are. The main thing I will be looking for are if the relationship of each predictor in the model in relation to the response variable are preserved and how these relationships explain whether or not they can be considered a useful predictor for overall energy consumption. 

***
## Model Building

### Model Equation

**Equation:** 

$$Y = \beta_0 +  \beta_1X_1 + \beta_2X_2 + \beta_3X_3 + \beta_4X_4 + \epsilon   

Where, 
Y= Total.Primary.Energy.Consumption → Y 
    B1= Total.Fossil.Fuel.Production → X1 
    B2= Nuclear.Electric.Power.Production → X2 
    B3= Total.Renewable.Energy.Production → X3
    B4= Primary.Energy.Imports → X4

### Model Fitting

In [ ]:
##MLR model
mlr.mod <- lm(Consumption ~ Fossil + Nuclear + Renewable + Imports, predictor.response.data)

### Multicollinearity

In [ ]:
#Pairwise scatter plot & multicolinearity
ggpairs(predictor.response.data)
VIF(lm(Fossil ~ Nuclear + Renewable + Imports, data= predictor.response.data))
VIF(lm(Nuclear ~ Fossil + Renewable + Imports, data= predictor.response.data))
VIF(lm(Renewable ~ Fossil + Nuclear + Imports, data= predictor.response.data))
VIF(lm(Imports ~ Fossil + Nuclear + Renewable, data= predictor.response.data))
VIF(lm(Fossil ~ Renewable, data= predictor.response.data))
VIF(lm(Nuclear ~ Imports, data= predictor.response.data))

### Model Summary and Diagonostics

In [ ]:
# Summary & confidence intervals
summary(mlr.mod)
confint(mlr.mod, level=0.95)

#Anova (type I & II)
anova(mlr.mod)
Anova(mlr.mod, type = 2)

# Diagnostics: Residual Plots, Normality, etc.
plot(mlr.mod)

#Cooks distance 
cooksd <- cooks.distance(mlr.mod)
plot(cooksd, type = "h", main = "Cook's Distance", ylab = "Cook's Distance")
abline(h = 4 / length(cooksd), col = "red")  # threshold for influential points

In [ ]:
##Individual SLR models
#Fossil
fossil.mod <- lm(Consumption ~ Fossil, predictor.response.data)
summary(fossil.mod)
ggplot(predictor.response.data, aes(x= Fossil, y= Consumption)) +
  geom_point() + 
  geom_smooth(method = "lm", color = "orange") + 
  theme_bw()

#Nuclear
nuclear.mod <- lm(Consumption ~ Nuclear, predictor.response.data)
summary(nuclear.mod)
ggplot(predictor.response.data, aes(x= Nuclear, y= Consumption)) +
  geom_point() + 
  geom_smooth(method = "lm", color = "blue") + 
  theme_bw()

#Renewable
renewable.mod <- lm(Consumption ~ Renewable, predictor.response.data)
summary(renewable.mod)
ggplot(predictor.response.data, aes(x= Renewable, y= Consumption)) +
  geom_point() + 
  geom_smooth(method = "lm", color = "green") + 
  theme_bw()

#Imports
imports.mod <- lm(Consumption ~ Imports, predictor.response.data)
summary(imports.mod)
ggplot(predictor.response.data, aes(x= Imports, y= Consumption)) +
  geom_point() + 
  geom_smooth(method = "lm", color = "purple") + 
  theme_bw()

In [ ]:
#Partial regression plots (component + residual) 
crPlots(mlr.mod)

***
## Model Evaluation & Validation

### Documentation of Model Adjustments

In [ ]:
##MLR model effects on renewables (the problem child)
fossil.renew <- lm(Consumption ~ Fossil + Renewable, predictor.response.data)
summary(fossil.renew)
confint(fossil.renew)
Anova(fossil.renew, type= 2)

nuclear.renew <- lm(Consumption ~ Nuclear + Renewable, predictor.response.data)
summary(nuclear.renew)
confint(nuclear.renew)
Anova(nuclear.renew, type= 2)

imports.renew <- lm(Consumption ~ Imports + Renewable, predictor.response.data)
summary(imports.renew)
confint(nuclear.renew)
Anova(imports.renew, type= 2)

In [ ]:
##log transformation 
ylog.mod = lm(log(Consumption)~Fossil+Nuclear+Renewable+Imports, 
              data = predictor.response.data)
summary(ylog.mod)
plot(ylog.mod)

In [ ]:
##WLS 
wts1 <- 1/fitted(lm(abs(residuals(mlr.mod)) ~ Fossil+Nuclear+Renewable+Imports,
                    predictor.response.data))^2
wls.mod = lm(Consumption ~ Fossil+Nuclear+Renewable+Imports, weight=wts1, 
             predictor.response.data)
summary(wls.mod)
confint(wls.mod)
plot(wls.mod)
bptest(mlr.mod)
bptest(wls.mod)

In [ ]:
##Bootstrapping
set.seed(400)
mlr = function(predictor.response.data, indices) {
  mlr_boot_data = predictor.response.data[indices, ]
  mlr_fit = lm(Consumption~Fossil+Nuclear+Renewable+Imports, data= mlr_boot_data)
  coefficients(mlr_fit)
}
mlr_boot = boot(data= predictor.response.data, statistic = mlr, R=100)
mlr_boot
boot.ci(mlr_boot, index= 2, type = "perc")
boot.ci(mlr_boot, index= 3, type = "perc")
boot.ci(mlr_boot, index= 4, type = "perc")
boot.ci(mlr_boot, index= 5, type = "perc")

In [ ]:
##Model selection with AIC 
aic = AIC(mlr.mod)
aic.wls = AIC(wls.mod)
aic.log = AIC(ylog.mod)
cat(aic)
cat(aic.wls)
cat(aic.log)

Summary of iterative process:
1. First I did a log transformation on the response variable (Consumption)
2. Then I did a WLS model to see its effect on model performance and fit of data
3. Then I did a boostrap resampling method 
4. Finally I compared AIC scores to pick the model that best balences goodness of fit and complexity

Final Model Equation: 
$$Y = \beta_0 +  \beta_1X_1 + \beta_2X_2 + \beta_3X_3 + \beta_4X_4 + \epsilon  


Where, 
Y= Total.Primary.Energy.Consumption → Y 
    B1= Total.Fossil.Fuel.Production → X1 
    B2= Nuclear.Electric.Power.Production → X2 
    B3= Total.Renewable.Energy.Production → X3
    B4= Primary.Energy.Imports → X4


I ended up going with the WLS model because the relationships between the predictor and response variables are preserved such as T-test p-values and direction and magnitude of coeffiecients. The weighted model also sligtly increased the R-squared value and AIC metrics show ΔAIC=6, so there is moderate evidence that the WLS model is better than the original using this method. In addition, the original structure of the model including the same response and predictor values was preserved so there is no substantial change to the model equation. 

From here, I will cross validate the WLS model using a k-fold method.

### Model Evaluation

### Validation Findings

In [ ]:
#cross-cross validation of WLS with K-fold where K=10
#method 1
#Define the training control
train_control <- trainControl(method = "cv", number = 10)  # 10-fold CV
#Train the model
trained.model <- train(Consumption ~ Fossil+Nuclear+Renewable+Imports,
                       weights= wts1, 
                       predictor.response.data, 
                       method = "lm", trControl = train_control)
#Print the model summary
print(trained.model)
summary(trained.model)

In [ ]:
#Save datafram as CSV file
write.csv(predictor.response.data, "model_data.csv")